<a href="https://colab.research.google.com/github/Mahdi-Golizadeh/Natural-Language-Processing/blob/main/transformers/token_classification/Token_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install & Import necessary libraries

In [1]:
!pip install datasets
!pip install transformers
!pip install poseval
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 11.7 MB/s 
     |████████████████████████████████| 212 kB 30.1 MB/s 
     |████████████████████████████████| 132 kB 32.9 MB/s 
     |████████████████████████████████| 182 kB 31.8 MB/s 
     |████████████████████████████████| 127 kB 48.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 15.3 MB/s 
     |████████████████████████████████| 7.6 MB 57.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement poseval (from versions: none)
ERROR: No matching distribution found for pos

In [2]:
import datasets
import transformers
import evaluate
import numpy as np

## Dataset

In [3]:
raw_datasets = datasets.load_dataset("conll2003")

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [5]:
raw_datasets["train"][0]["tokens"], raw_datasets["train"][0]["pos_tags"]

(['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'],
 [22, 42, 16, 21, 35, 37, 16, 21, 7])

In [6]:
pos_features = raw_datasets["train"].features["pos_tags"]

In [7]:
pos_features

Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None)

In [8]:
label_names = pos_features.feature.names

In [9]:
names = pos_features.feature.names
words = raw_datasets["train"][5]["tokens"]
labels = raw_datasets["train"][5]["pos_tags"]
l_1 = ""
l_2 = ""
for word, label in zip(words, labels):
    label_name = names[label]
    max_length = max(len(word), len(label_name))
    l_1 += word + " " * (max_length - len(word) + 1)
    if label_name in ['"', "''", '#',  '$',  '(',  ')', ',', '.', ':', '``',]:
        l_2 +=" " * (max_length + 1)
    else:
        l_2 += label_name + " " * (max_length - len(label_name) + 1)

print(l_1, l_2, sep= "\n")

" We  do  n't support any such recommendation because we  do  n't see any grounds for it  , " the Commission 's  chief spokesman Nikolaus van der Pas told a  news briefing . 
  PRP VBP RB  VB      DT  JJ   NN             IN      PRP VBP RB  VB  DT  NNS     IN  PRP     DT  NNP        POS JJ    NN        NNP      NNP FW  NNP VBD  DT NN   NN         


In [10]:
checkpoint = "bert-base-cased"

In [11]:
tokenizer = transformers.AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [12]:
tokenizer.is_fast


True

In [13]:
inputs = tokenizer(raw_datasets["train"][2]["tokens"],
                   is_split_into_words= True)

In [14]:
inputs

{'input_ids': [101, 26660, 13329, 12649, 15928, 1820, 118, 4775, 118, 1659, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
inputs.tokens()

['[CLS]', 'BR', '##US', '##SE', '##LS', '1996', '-', '08', '-', '22', '[SEP]']

In [16]:
inputs.word_ids()

[None, 0, 0, 0, 0, 1, 1, 1, 1, 1, None]

we are required to convert special tokens into -100 in order to be ignored by loss function

In [17]:
def special_token_care(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            new_labels.append(-100)
        else:
            label = labels[word_id]
    return new_labels

In [18]:
def tokenize_and_care(example):
    tokenized_input = tokenizer(example["tokens"],
                                truncation= True,
                                is_split_into_words= True)
    all_labels = example["pos_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_input.word_ids(i)
        new_labels.append(special_token_care(labels, word_ids))
        tokenized_input["labels"] = new_labels
    return tokenized_input

In [19]:
tokenized_datasets = raw_datasets.map(tokenize_and_care, 
                                      batched= True,
                                      remove_columns= raw_datasets["train"].column_names)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [20]:
tokenized_datasets["train"][12]

{'input_ids': [101,
  2809,
  1699,
  1105,
  2855,
  5534,
  17355,
  9022,
  2879,
  112,
  188,
  5835,
  119,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 30, 22, 10, 22, 38, 22, 27, 21, 7, -100]}

In [21]:
data_collator = transformers.DataCollatorForTokenClassification(tokenizer= tokenizer)

In [22]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [23]:
batch["input_ids"]

tensor([[  101,  7270, 22961,  1528,  1840,  1106, 21423,  1418,  2495, 12913,
           119,   102],
        [  101,  1943, 14428,   102,     0,     0,     0,     0,     0,     0,
             0,     0]])

In [24]:
metric = evaluate.load("poseval")

In [25]:
metric

EvaluationModule(name: "poseval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Computes the poseval metric.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    zero_division: Which value to substitute as a metric value when encountering zero division. Should be on of 0, 1,
        "warn". "warn" acts as 0, but the warning is raised.

Returns:
    'scores': dict. Summary of the scores for overall and per type
        Overall (weighted and macro avg):
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': F1 score, also known as balanced F-score or F-measure,
        Per type:
            'precision': p

In [26]:
def compute_metric(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis= -1)
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [[label_names[p] for (p,l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    metrics = metric.compute(predictions= true_predictions,
                             reference= true_labels)
    return {"precision": metrics["weighted avg"]["precision"],
            "recall": metrics["weighted avg"]["recall"],
            "f1":metrics["weighted avg"]["f1_score"],}

In [27]:
labels = raw_datasets["train"][0]["pos_tags"]
labels = [label_names[i] for i in labels]
predictions = labels.copy()
predictions[2] = "VB"
result = metric.compute(predictions= [predictions],
               references= [labels])

In [28]:
result["weighted avg"]

{'precision': 0.9444444444444444,
 'recall': 0.8888888888888888,
 'f1-score': 0.8888888888888888,
 'support': 9}

In [29]:
id2labels = {i: label for i, label in enumerate(label_names)}
label2ids = {v: k for k, v in id2labels.items()}

In [30]:
model = transformers.AutoModelForTokenClassification.from_pretrained(checkpoint,
                                                                     id2label=id2labels,
                                                                     label2id= label2ids)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [31]:
model.config.num_labels

47

In [32]:
len(id2labels)

47

In [33]:
args = transformers.TrainingArguments("bert-finetuned-pos",
                                      evaluation_strategy= "epoch",
                                      save_strategy= "epoch",
                                      learning_rate= 2e-5,
                                      num_train_epochs= 5,
                                      weight_decay= .01)

In [34]:
trainer = transformers.Trainer(model= model,
                               args= args,
                               train_dataset= tokenized_datasets["train"],
                               eval_dataset= tokenized_datasets["validation"],
                               data_collator= data_collator,
                               compute_metrics= compute_metric,
                               tokenizer= tokenizer)

In [35]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14041
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 8780
  Number of trainable parameters = 107755823


Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 3250
  Batch size = 8


TypeError: ignored